In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
import importlib.util; print(importlib.util.find_spec('transformers').origin)


/Users/Zhuanz/works/transformers/src/transformers/__init__.py


In [3]:
import pickle
l1_result_bf16 = pickle.load(open("./qw2.7B.l1.bf16.pkl", "rb"))
l1_result_fp32 = pickle.load(open("./qw2.7B.l1.fp32.pkl", "rb"))

In [5]:
l1_result_fp32

{(0, 0, 'gate'): 39375.2109375,
 (0, 0, 'up'): 39354.22265625,
 (0, 0, 'down'): 39407.4765625,
 (0, 1, 'gate'): 39274.1015625,
 (0, 1, 'up'): 39077.86328125,
 (0, 1, 'down'): 39362.49609375,
 (0, 2, 'gate'): 38983.82421875,
 (0, 2, 'up'): 38418.9765625,
 (0, 2, 'down'): 38814.39453125,
 (0, 3, 'gate'): 38139.46875,
 (0, 3, 'up'): 37297.0078125,
 (0, 3, 'down'): 38060.67578125,
 (0, 4, 'gate'): 38704.83203125,
 (0, 4, 'up'): 37979.30859375,
 (0, 4, 'down'): 38555.703125,
 (0, 5, 'gate'): 32694.86328125,
 (0, 5, 'up'): 30375.748046875,
 (0, 5, 'down'): 32666.30859375,
 (0, 6, 'gate'): 36044.86328125,
 (0, 6, 'up'): 34619.140625,
 (0, 6, 'down'): 36627.0078125,
 (0, 7, 'gate'): 39328.390625,
 (0, 7, 'up'): 38719.72265625,
 (0, 7, 'down'): 39026.40625,
 (0, 8, 'gate'): 39295.96484375,
 (0, 8, 'up'): 39270.890625,
 (0, 8, 'down'): 39407.625,
 (0, 9, 'gate'): 37854.93359375,
 (0, 9, 'up'): 37699.7734375,
 (0, 9, 'down'): 38318.00390625,
 (0, 10, 'gate'): 39106.83203125,
 (0, 10, 'up'): 38629

In [9]:
values = []
for key, value in l1_result_fp32.items():
    values.append((key[0], key[1], key[2], value))
import pandas as pd
df = pd.DataFrame(values, columns=["layer", "expert", "var", "l1"])
df

/Users/Zhuanz/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/Zhuanz/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


,layer,expert,var,l1
0,0,0,gate,39375.210938
1,0,0,up,39354.222656
2,0,0,down,39407.476562
3,0,1,gate,39274.101562
4,0,1,up,39077.863281
...,...,...,...,...
4603,23,62,up,35859.164062
4604,23,62,down,29220.931641
4605,23,63,gate,33104.187500
4606,23,63,up,36033.621094


In [13]:
df = df[df["var"]=="down"]
expert_order_output = []
for layer_id in range(24):
    print("layer {}".format(layer_id))
    tmp_df = df[df["layer"]==layer_id]
    tmp_df = tmp_df.sort_values("l1")
    expert_idxs = list(tmp_df["expert"])
    l1_list = list(tmp_df["l1"])
    print(expert_idxs)
    print(l1_list)
    print("min l1 {}; max l1 {}".format(min(l1_list), max(l1_list)))
    expert_order_output.append(expert_idxs)

layer 0
[45, 37, 43, 5, 42, 33, 6, 31, 38, 39, 28, 26, 18, 59, 51, 58, 36, 3, 34, 55, 50, 19, 9, 53, 41, 46, 60, 62, 61, 63, 32, 25, 4, 49, 23, 2, 40, 20, 17, 7, 11, 10, 56, 27, 21, 57, 14, 12, 54, 30, 1, 0, 8, 29, 52, 35, 47, 22, 15, 24, 16, 48, 44, 13]
[28926.140625, 29352.27734375, 32170.9296875, 32666.30859375, 35951.8046875, 36483.8671875, 36627.0078125, 36705.15625, 36820.44921875, 36921.57421875, 36933.50390625, 36941.953125, 37098.8984375, 37203.83984375, 37439.96875, 37547.74609375, 37594.10546875, 38060.67578125, 38087.46875, 38111.1953125, 38115.609375, 38211.40625, 38318.00390625, 38337.7890625, 38338.85546875, 38348.34765625, 38365.28515625, 38413.09375, 38481.34765625, 38498.546875, 38513.890625, 38554.35546875, 38555.703125, 38640.59375, 38744.5, 38814.39453125, 38857.8671875, 38916.05859375, 38942.44921875, 39026.40625, 39039.16796875, 39084.25, 39136.765625, 39175.234375, 39177.4453125, 39229.234375, 39250.53515625, 39260.51171875, 39318.00390625, 39323.078125, 39362.4

In [20]:
layer_idx_to_expert_idx = {}
for i, expert_idxs in enumerate(expert_order_output):
    layer_idx_to_expert_idx[i] = expert_idxs
print(layer_idx_to_expert_idx)
import json
json.dump(layer_idx_to_expert_idx, open("./moe_prune/layer_idx_to_expert_idx.json", 'w'))

{0: [45, 37, 43, 5, 42, 33, 6, 31, 38, 39, 28, 26, 18, 59, 51, 58, 36, 3, 34, 55, 50, 19, 9, 53, 41, 46, 60, 62, 61, 63, 32, 25, 4, 49, 23, 2, 40, 20, 17, 7, 11, 10, 56, 27, 21, 57, 14, 12, 54, 30, 1, 0, 8, 29, 52, 35, 47, 22, 15, 24, 16, 48, 44, 13], 1: [42, 0, 56, 23, 45, 29, 48, 51, 21, 10, 37, 16, 24, 40, 13, 9, 59, 27, 38, 26, 8, 3, 31, 32, 33, 53, 41, 57, 18, 14, 34, 2, 43, 58, 20, 30, 52, 4, 7, 44, 35, 39, 46, 6, 19, 11, 12, 36, 55, 28, 62, 60, 1, 61, 5, 17, 47, 22, 15, 25, 50, 49, 63, 54], 2: [57, 0, 54, 59, 23, 52, 4, 62, 41, 27, 18, 60, 13, 42, 61, 40, 1, 20, 22, 47, 9, 21, 39, 55, 16, 19, 53, 3, 51, 14, 7, 63, 2, 37, 50, 30, 44, 56, 12, 15, 31, 25, 38, 28, 33, 36, 46, 32, 34, 5, 26, 45, 11, 10, 43, 49, 17, 8, 48, 6, 58, 35, 29, 24], 3: [33, 9, 19, 51, 63, 60, 61, 47, 2, 40, 22, 62, 41, 32, 39, 45, 35, 26, 34, 31, 0, 15, 56, 29, 30, 10, 48, 18, 3, 57, 4, 16, 44, 12, 43, 37, 7, 5, 13, 23, 59, 25, 58, 11, 53, 6, 1, 54, 21, 27, 55, 50, 17, 24, 38, 46, 42, 8, 28, 52, 14, 20, 36, 

alpha

In [3]:
import json
layer_to_alpha = json.load(open("data/layer_idx_to_alpha_list.json", 'r'))

In [4]:
layer_to_alpha

{'0': [7.491056274772742,
  7.7005345744798745,
  6.175452662225098,
  5.811440539691987,
  4.925766416844041,
  3.552588152228438,
  3.895943996875954,
  6.026037576412072,
  7.545960228970823,
  5.589701562606153,
  6.420193576079267,
  6.82085439336948,
  11.324474208165952,
  8.18756435615591,
  6.1791679777760224,
  8.184519990955815,
  7.9939559017869115,
  7.422498799405636,
  4.728031594528442,
  5.556432308165707,
  6.118321444606121,
  10.7470376966485,
  8.594315210915898,
  6.75616298177125,
  8.83032009601906,
  4.705698327700606,
  4.862998678727159,
  7.591160895235023,
  4.942120895975728,
  8.721779184801614,
  7.946088122997455,
  3.7275432921104654,
  5.895990449900878,
  3.3252809164451507,
  5.423898269206735,
  8.41801961534207,
  4.243375783222825,
  3.523047302120067,
  4.365300256298408,
  5.0357476838529,
  5.000952269576394,
  5.536528713251513,
  4.458178197011377,
  4.252813127707274,
  8.430168642038367,
  3.157250948506735,
  5.2068058489753195,
  7.88655

In [ ]:
layer_idx_to_expert_idx_alpha = {}
for layer_idx in layer_to_alpha:
    alphas = layer_to_alpha[layer_idx]
    expert_idx_with_alpha = list(enumerate(alphas))